## Exporting CSVs from the FairNow APIs

#### This generates a CVS export of the Compliance report, containing applications, and frameworks.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [9]:
import pandas as pd
from utils.fairnow import get_client

client = get_client()

# from getpass import getpass
# import httpx
# from httpx_auth import OAuth2ClientCredentials

# client_id = getpass("{client_id}") # Replace with your Client Id
# client_secret = getpass("Client Secret")
# fairnow_token_endpoint = "https://auth.fairnow.dev/oauth2/token"

# auth = OAuth2ClientCredentials(
#     token_url=fairnow_token_endpoint,
#     client_id=client_id,
#     client_secret=client_secret,
# )

# fairnow_base_url = "https://api.fairnow.dev/v2"
# client = httpx.Client(base_url=fairnow_base_url, auth=auth)

## Create the Applications DataFrame

In [ ]:
application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

# Extract fields from response
extracted_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    risk_assessment = app.get('risk_assessment', {})
    framework_items = risk_assessment.get('framework_assessment_items', []) or []
    for framework in framework_items:
        extracted_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'framework_id': framework.get('framework_id', ''),
            'framework_name': framework.get('framework_name', ''),
        })

# Convert to DataFrame
apps_df = pd.DataFrame(extracted_data)
apps_df = apps_df.drop_duplicates()
print(f"Extracted frameworks")

## Frameworks API Call

In [21]:
def get_framework_controls(framework_id):
    application_route = f"/controls/framework/"
    query_parameters = {"framework_id": framework_id}
    framework_response = None
    
    try:
        response = client.get(application_route, params=query_parameters, timeout=None)
        if response.status_code == 200:
            framework_response = response.json()
            print(f"Successfully retrieved framework controls")
            return framework_response
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(e)
        return None

## Get Framework IDs for each entry in Applications DataFrame

In [ ]:
framework_ids = apps_df['framework_id'].unique()

controls_data = []
# Get controls for each framework
for framework_id in framework_ids:
    controls = get_framework_controls(framework_id)
    if controls:
        # Add framework_id to each control record
        for control in controls:
            control['framework_id'] = framework_id
        controls_data.extend(controls)

# Create a DataFrame of controls data
controls_df = pd.DataFrame(controls_data)
controls_df = controls_df.drop_duplicates()
print(f"Extracted controls")

## Merge DataFrames

In [ ]:
# Merge with the original DataFrame
# This will add the control information to each framework row
merged_df = pd.merge(
    apps_df[['application_id', 'application_name', 'framework_id', 'framework_name']],
    controls_df[['framework_id', 'applications_count', 'applications_ready']],
    on='framework_id',
    how='left'
)
print(f"Extracted {len(merged_df)} records")

### Export to TSV

In [ ]:
# Export to TSV
if 'merged_df':
    output_file = 'compliance_export.tsv'
    merged_df.to_csv(output_file, sep='\t', index=False)
    print(f"\nData exported to {output_file}")
else:
    print("No data to export")